In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE132903"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE132903"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE132903.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE132903.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE132903.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome changes in the Alzheimer's middle temporal gyrus: importance of RNA metabolism and mitochondria-associated membrane (MAM) genes"
!Series_summary	"We used Illumina Human HT-12 v4 arrays to compare RNA expression of middle temporal gyrus (MTG; BA21) in Alzheimer’s Disease (AD = 97) and non-demented controls (ND = 98).  A total of 938 transcripts were highly differentially expressed (adj p < 0.01; log2 Fold Change (FC) ≥ |0.500|, with 411 overexpressed and 527 underexpressed in AD.  Our results correlated with expression profiling in neurons from AD and ND obtained by Laser Capture Microscopy in MTG  from an independent dataset (log2 FC correlation: r = 0.504; p = 2.2e-16).  Additionally selected effects were validated by qPCR. ANOVA analysis yielded no difference between genders in response to AD, but some gender specific genes were detected (e.g: IL8 and AGRN in males, and HSPH1 and GRM1 in females).  Several transcripts were associat

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Determine gene expression data availability
# From the background info, we see Illumina Human HT-12 v4 arrays were used
# for RNA expression, which indicates gene expression data.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Alzheimer's Disease), we can use the 'diagnosis' field
trait_row = 3  # 'diagnosis: ND', 'diagnosis: AD'

# For age, we have 'expired_age (years)' field
age_row = 2  # Contains ages of participants

# For gender, we have 'Sex' field
gender_row = 1  # 'Sex: female', 'Sex: male'

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert diagnosis to binary trait value."""
    if not value or ':' not in value:
        return None
    
    diagnosis = value.split(':', 1)[1].strip()
    if diagnosis == 'AD':
        return 1  # Alzheimer's Disease
    elif diagnosis == 'ND':
        return 0  # Non-demented control
    else:
        return None

def convert_age(value):
    """Convert age string to continuous numeric value."""
    if not value or ':' not in value:
        return None
    
    age_str = value.split(':', 1)[1].strip()
    try:
        if age_str.endswith('+'):
            # For 90+, use 90 as the base age
            return float(age_str.replace('+', ''))
        else:
            return float(age_str)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender string to binary."""
    if not value or ':' not in value:
        return None
    
    gender = value.split(':', 1)[1].strip().lower()
    if gender == 'female':
        return 0
    elif gender == 'male':
        return 1
    else:
        return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# We have only the sample characteristics dictionary showing unique values,
# but we need the actual clinical data to perform the extraction.
# We'll print the information we've gathered for debugging purposes
if trait_row is not None:
    print(f"Trait row identified: {trait_row}")
    print(f"Age row identified: {age_row}")
    print(f"Gender row identified: {gender_row}")
    print("The actual clinical data extraction requires the full clinical dataset.")
    print("We've identified the relevant rows and created the conversion functions.")
    
    # Since we can't perform the actual extraction without the clinical data,
    # we'll just note that we've saved the metadata indicating the dataset
    # has the necessary trait information
    print(f"Metadata saved to {json_path}, indicating trait data is available: {is_trait_available}")


Trait row identified: 3
Age row identified: 2
Gender row identified: 1
The actual clinical data extraction requires the full clinical dataset.
We've identified the relevant rows and created the conversion functions.
Metadata saved to ../../output/preprocess/Alzheimers_Disease/cohort_info.json, indicating trait data is available: True


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651249', 'ILMN_1651254', 'ILMN_1651259',
       'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers
# The identifiers follow the "ILMN_" prefix pattern, which indicates they are Illumina probe IDs
# These are not human gene symbols but Illumina BeadArray probe identifiers that need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns to use for gene identifier mapping
# From the gene annotation preview, 'ID' column contains the probe identifiers matching the gene expression data index
# The 'Symbol' column contains the gene symbols we want to map to
probe_col = 'ID'
gene_col = 'Symbol'

# 2. Get the gene mapping dataframe by extracting the identifier and symbol columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Print first few rows of the mapping
print("Gene mapping (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few genes 
print("\nGene expression data after mapping (first 5 genes):")
print(gene_data.index[:5])
print(f"Total number of genes after mapping: {len(gene_data)}")


Gene mapping (first 5 rows):
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB

Gene expression data after mapping (first 5 genes):
Index(['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2M'], dtype='object', name='Gene')
Total number of genes after mapping: 19788


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...


Gene data shape after normalization: (18799, 195)


Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE132903.csv
Loading the original clinical data...


Extracting clinical features...
Clinical data preview:
{'GSM3895951': [0.0, 90.0, 0.0], 'GSM3895952': [0.0, 82.0, 1.0], 'GSM3895953': [0.0, 88.0, 0.0], 'GSM3895954': [0.0, 92.0, 0.0], 'GSM3895955': [0.0, 91.0, 1.0], 'GSM3895956': [0.0, 87.0, 0.0], 'GSM3895957': [0.0, 86.0, 1.0], 'GSM3895958': [0.0, 78.0, 1.0], 'GSM3895959': [0.0, 87.0, 1.0], 'GSM3895960': [0.0, 79.0, 1.0], 'GSM3895961': [0.0, 77.0, 0.0], 'GSM3895962': [0.0, 77.0, 1.0], 'GSM3895963': [0.0, 88.0, 0.0], 'GSM3895964': [0.0, 85.0, 1.0], 'GSM3895965': [0.0, 95.0, 0.0], 'GSM3895966': [0.0, 102.0, 0.0], 'GSM3895967': [0.0, 89.0, 1.0], 'GSM3895968': [0.0, 70.0, 1.0], 'GSM3895969': [0.0, 82.0, 0.0], 'GSM3895970': [0.0, 73.0, 0.0], 'GSM3895971': [0.0, 90.0, 1.0], 'GSM3895972': [0.0, 94.0, 1.0], 'GSM3895973': [0.0, 96.0, 0.0], 'GSM3895974': [0.0, 85.0, 0.0], 'GSM3895975': [0.0, 84.0, 1.0], 'GSM3895976': [0.0, 83.0, 1.0], 'GSM3895977': [0.0, 90.0, 0.0], 'GSM3895978': [0.0, 87.0, 0.0], 'GSM3895979': [0.0, 85.0, 1.0], 'GSM3895980': [

Linked data shape after handling missing values: (195, 18802)
Checking for bias in trait distribution...
For the feature 'Alzheimers_Disease', the least common label is '1.0' with 97 occurrences. This represents 49.74% of the dataset.
The distribution of the feature 'Alzheimers_Disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 80.5
  50% (Median): 85.0
  75%: 90.0
Min: 70.0
Max: 102.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 96 occurrences. This represents 49.23% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Alzheimers_Disease/GSE132903.csv
